In [1]:
# Import dependencies
import mysql.connector
import json
import sqlalchemy

# read database config file
with open("parameters/config.json") as config:
    param = json.load(config)

# settting up the function with parameters
def dataupdate(MetricId, DataPoint, EffectiveBeginDate, EffectiveEndDate, CreatedDate,CreatedBy):
    # Connecting to data warehouse
    try:
        engine = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                          format(param['Teletron'][0]['user'],
                                                 param['Teletron'][0]['password'],
                                                 param['Teletron'][0]['host'],
                                                 param['Teletron'][0]['database']), echo=False)

        # Cleaning the data from existing tables MetricValues and Metrics
        Epi_con = engine.connect()
        # If connection is established, push the data into datawarehouse
        if Epi_con.connect():
            try:
                sql = """INSERT INTO SlavsTest (MetricId, DataPoint, EffectiveBeginDate, EffectiveEndDate, CreatedDate,CreatedBy)
                            VALUES (%s, %s, %s, %s,%s,%s) """

                record = (MetricId, DataPoint, EffectiveBeginDate, EffectiveEndDate, CreatedDate,CreatedBy)
                Epi_con.execute(sql,record)
                Epi_con.close()
                engine.dispose()
            except OSError as e:
                print(e)
    except OSError as e:
        print(e)


In [11]:
# Importing Dependencies
import mysql.connector
import pandas as pd
import json
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table
from mysql.connector import Error

sample = pd.read_csv("data/Sample.csv")

sample

,Id,RecordDate,Temperature,Pressure,CreatedBy
0,1,2/23/2022,75,1000,Slav


In [14]:

with open("parameters/config.json") as config:
    param = json.load(config)

# Connecting to data warehouse
    try:
        engine = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                          format(param['Teletron'][0]['user'],
                                                 param['Teletron'][0]['password'],
                                                 param['Teletron'][0]['host'],
                                                 param['Teletron'][0]['database']), echo=False)

        # Cleaning the data from existing tables MetricValues and Metrics

        Teletron_con = engine.connect()
        Teletron_con.execute("""TRUNCATE TABLE crudexample""")
        # If connection is established, push the data into datawarehouse
        if Teletron_con.connect():
            try:
                sample.to_sql('crudexample', con=Teletron_con, if_exists='append',chunksize=1000, index=False)
                Teletron_con.close()
                engine.dispose()
            except OSError as e:
                print(e)
    except OSError as e:
                print(e)


C:\Users\slavo\PycharmProjects\Python-Crud-Application\venv\lib\site-packages\pandas\io\sql.py:1663: UserWarning: The provided table name 'Crud' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [16]:
from pathlib import Path
from itertools import islice

def tree(dir_path: Path, level: int=-1, limit_to_directories: bool=False,length_limit: int=1000):
    # prefix components:
    space = '    '
    branch = '│   '
    # pointers:
    tee = '├── '
    last = '└── '
    """Given a directory Path object print a visual tree structure"""
    dir_path = Path(dir_path) # accept string coerceable to Path
    files = 0
    directories = 0
    def inner(dir_path: Path, prefix: str='', level=-1):
        nonlocal files, directories
        if not level:
            return # 0, stop iterating
        if limit_to_directories:
            contents = [d for d in dir_path.iterdir() if d.is_dir()]
        else:
            contents = list(dir_path.iterdir())
        pointers = [tee] * (len(contents) - 1) + [last]
        for pointer, path in zip(pointers, contents):
            if path.is_dir():
                yield prefix + pointer + path.name
                directories += 1
                extension = branch if pointer == tee else space
                yield from inner(path, prefix=prefix+extension, level=level-1)
            elif not limit_to_directories:
                yield prefix + pointer + path.name
                files += 1
    print(dir_path.name)
    iterator = inner(dir_path, level=level)
    for line in islice(iterator, length_limit):
        print(line)
    if next(iterator, None):
        print(f'... length_limit, {length_limit}, reached, counted:')
    print(f'\\n{directories} directories' + (f', {files} files' if files else ''))

tree(Path.home() / 'PycharmProjects/Python-Crud-Application', level=2)

Python-Crud-Application
├── .git
│   ├── config
│   ├── description
│   ├── HEAD
│   ├── hooks
│   ├── index
│   ├── info
│   ├── logs
│   ├── objects
│   ├── packed-refs
│   └── refs
├── .gitignore
├── .idea
│   ├── .gitignore
│   ├── DataEntryApp.iml
│   ├── inspectionProfiles
│   ├── misc.xml
│   ├── modules.xml
│   ├── Python-Crud-Application.iml
│   ├── vcs.xml
│   └── workspace.xml
├── data
│   ├── CRUD-1.png
│   ├── CRUD.jpeg
│   ├── CRUD.png
│   └── Sample.csv
├── GUI.py
├── lib
│   ├── DataImport.py
│   ├── Tools.py
│   └── __pycache__
├── LICENSE
├── main.py
├── notebooks
│   └── Dev.ipynb
├── parameters
│   └── config.json
├── README.md
└── venv
    ├── .gitignore
    ├── etc
    ├── include
    ├── Lib
    ├── pyvenv.cfg
    ├── Scripts
    └── share
\n19 directories, 27 files
